In [1]:
import ujson, csv, time
from dtran import Pipeline, IFunc, ArgType
from funcs import ReadFunc
from pydrepr import Graph

%load_ext autoreload
%autoreload 2

# This is a demo with an example flow for the MINT transformation pipeline

This notebook presents an abstraction of the MINT Data Transformations architecture and walks thourgh an example through the pipeline we are suggesting; we show a simple CSV --(unit transformations)--> CSV flow.

# (Semantic) Description of a component (reader/writer/transformation functions)

Providing semantic description of a function may allow constructing the pipeline semi-automatically. There are two levels of a semantic description:

1. Providing definition for inputs/outputs of a component: it enables input data validation and compatibility checking between connected components.
2. Providing definition of the purpose of the component: it allows us to construct the transformation pipeline based on some specification from users

```python
class IFunc(abc.ABC):
    id: str = ""
        
    # level 2
    description = None
    
    # level 1
    inputs: Dict[str, ArgType] = {}
    outputs: Dict[str, ArgType] = {}

    @abc.abstractmethod
    def validate(self) -> bool:
        """
        Check if the inputs are correct or not
        :return:
        """
        raise NotImplementedError()

    @abc.abstractmethod
    def exec(self) -> dict:
        """
        Execute the transformation function and return the result
        :return:
        """
        raise NotImplementedError()
```

## This is the semantic description of the interface (control plane) between all of the components in the pipeline (Reader/Writer/Transformation Adapters)


## Here's an example flow:

## Component 1: Reader Adapter

An instance of a reader adapter can be used as an entry point in the pipeline. It reads an input file `input.csv` file and a `input.yaml` file describing the D-REPR layout of this file. The data are representated in general way in a python object (Graph or NumPY array) and will be used in the next steps in the pipeline.

```python
class ReadFunc(IFunc):
    id = "read_func"
    inputs = {
        "repr_file": ArgType.FilePath, 
        "resources": ArgType.String
    }
    outputs = {"data": ArgType.Graph(None)}
    
    ...
```

## Component 2: Transformation Adapter

An instance of a transformation adapter does not materialize the data into an output, it just reproduces the data, transformaing its content (the actual data) and performing the needed configrations in the 'control plane' parameters.
Given the pythonic object (graph) representing the data and the rest of the needed configurations in the calling API ('control plane' parameters) we can re-construct the data in a new pythonic object (graph) and prepare it for the next steps in the pipeline.

In [2]:
# python_graph_obj --> python_graph_obj*
class UnitTransFunc(IFunc):
    id = "unit_trans"
    inputs = {"graph": ArgType.Graph(None)}
    outputs = {"graph": ArgType.Graph(None)}
    
    def __init__(self, graph: Graph):
        self.graph = graph
        
    def exec(self):
        return {
            "graph": Graph({}, [], [])
        }

## Component 3: Writer Adapter

An instance of a writer adapter can be used as an exit point in the pipeline. It writes an output file `output.csv` based on a given `output.yaml` (D-REPR layout) and an additional configuration file `output.config`

In [8]:
# {python_graph_obj*, output.yaml, output.config} --> {output.csv}
class WriteFunc(IFunc):
    id = "write_func"
    inputs = {}
    outputs = {}

# Pipeline

In [4]:
inputs = {
    ReadFunc.I.repr_file: "./examples/s01_ethiopia_commodity_price.yml",
    ReadFunc.I.resources: "./examples/s01_ethiopia_commodity_price.csv",
}

pipeline = Pipeline([
    ReadFunc,
    UnitTransFunc
], wired=[
    ReadFunc.O.data == UnitTransFunc.I.graph
])
outputs = pipeline.exec(inputs)

In [10]:
outputs[str(ReadFunc.O._0.data)]